# Simulating Vessel Operations and Logistics at a Shipping Terminal Using SimPy
#### Note: Manual Control Over Resource Release, Resource requests made without using 'with' satement

#### Using same arguments

In [2]:
import simpy
import random

#### Resource Variables

In [3]:
AVG_ARRIVAL_TIME = 5 * 60  # Average arrival time in minutes (5 hours)
CONTAINER_COUNT = 150 # Provided number of containers in each vessel
CRANE_MOVE_TIME = 3  # Crane move time per container in minutes
TRUCK_ROUND_TRIP_TIME = 6  # Truck round trip time in minutes
SIMULATION_TIME = 24 * 60  # Simulation time in minutes (24 hours)

#### Time Coversion function

In [4]:
def convert_minutes_to_hhmm(minutes):
    hours = int(minutes // 60)
    minutes = int(minutes % 60)
    return hours, minutes

#### Main Function

In [5]:
def vessel(env, name, terminal):

    #log event: 1 --> Vessel Arrival
    arrival_time = env.now
    arrival_hours, arrival_minutes = convert_minutes_to_hhmm(arrival_time)
    print(f"{name} arrives at {arrival_hours} hours and {arrival_minutes} minutes")
    
    # Request berth
    berth_req = terminal.berths.request()
    yield berth_req

    #log event: 2 --> When vessel get the berth
    berthing_time = env.now
    berthing_hours, berthing_minutes = convert_minutes_to_hhmm(berthing_time)

    #log the event: 2.1 --> waiting time to get the berth
    waiting_time = berthing_time - arrival_time
    waiting_hours, waiting_minutes = convert_minutes_to_hhmm(waiting_time)
    print(f"{name} berths at {berthing_hours} hours and {berthing_minutes} minutes after waiting for {waiting_hours} hours and {waiting_minutes} minutes")
    
    try:
        # Start unloading containers
        for i in range(CONTAINER_COUNT):

            # generating request for quay crane
            crane_req = terminal.cranes.request()
            yield crane_req

             #log the event: 3 --> when crane started unloading the particular container
            container_time = env.now
            container_hours, container_minutes = convert_minutes_to_hhmm(container_time)
            print(f"{name}: Crane starts unloading container {i+1} at {container_hours} hours and {container_minutes} minutes")
            
            # Move container
            yield env.timeout(CRANE_MOVE_TIME)
            
            # Place container on truck
            truck_req = terminal.trucks.request()
            truck_arrival_time = env.now
            truck_arrival_hours, truck_arrival_minutes = convert_minutes_to_hhmm(truck_arrival_time)

            #log event: 4 --> waiting time to get the truck
            waiting_for_truck_time = truck_arrival_time - (container_time + CRANE_MOVE_TIME)
            if waiting_for_truck_time > 0:
                waiting_for_truck_hours, waiting_for_truck_minutes = convert_minutes_to_hhmm(waiting_for_truck_time)
                print(f"{name}: Crane waits for truck for {waiting_for_truck_hours} hours and {waiting_for_truck_minutes} minutes")

            
            yield truck_req

            #log event: 4.1 --> Truck Arrival time
            truck_time = env.now
            truck_hours, truck_minutes = convert_minutes_to_hhmm(truck_time)
            print(f"{name}: Crane places container {i+1} on truck at {truck_hours} hours and {truck_minutes} minutes")
            
            # Transport container to yard block
            print(f"Truck starts transporting container {i+1} to yard block at {truck_hours} hours and {truck_minutes} minutes")
            yield env.timeout(TRUCK_ROUND_TRIP_TIME)

            #log event: 5 --> Truck returns to berth place from yard
            return_time = env.now
            return_hours, return_minutes = convert_minutes_to_hhmm(return_time)
            print(f"Truck returns to quay crane at {return_hours} hours and {return_minutes} minutes")
            
            # Release resources (truck and crane)
            terminal.cranes.release(crane_req)
            terminal.trucks.release(truck_req)
    
    finally:
        # Vessel departs
        departure_time = env.now
        departure_hours, departure_minutes = convert_minutes_to_hhmm(departure_time)
        print(f"{name} departs at {departure_hours} hours and {departure_minutes} minutes")
        
        # Release berth
        terminal.berths.release(berth_req)

#### Generator function

In [6]:
def vessel_arrival_generator(env, terminal):
    i = 0
    while True:
        # Yield a timeout for vessels which is following an exponential distribution
        yield env.timeout(random.expovariate(1 / AVG_ARRIVAL_TIME))
        env.process(vessel(env, f"Vessel {i+1}", terminal))
        i += 1

In [7]:
# Terminal Class --> initializing the resources
class Terminal:
    def __init__(self, env):
        self.env = env   
        self.berths = simpy.Resource(env, capacity=2)
        self.cranes = simpy.Resource(env, capacity=2)
        self.trucks = simpy.Resource(env, capacity=3)

## Satrt Simulation

In [8]:
# Simulation Setup
env = simpy.Environment()
terminal = Terminal(env)
env.process(vessel_arrival_generator(env, terminal))
env.run(until=SIMULATION_TIME)

Vessel 1 arrives at 17 hours and 43 minutes
Vessel 1 berths at 17 hours and 43 minutes after waiting for 0 hours and 0 minutes
Vessel 1: Crane starts unloading container 1 at 17 hours and 43 minutes
Vessel 1: Crane places container 1 on truck at 17 hours and 46 minutes
Truck starts transporting container 1 to yard block at 17 hours and 46 minutes
Truck returns to quay crane at 17 hours and 52 minutes
Vessel 1: Crane starts unloading container 2 at 17 hours and 52 minutes
Vessel 1: Crane places container 2 on truck at 17 hours and 55 minutes
Truck starts transporting container 2 to yard block at 17 hours and 55 minutes
Truck returns to quay crane at 18 hours and 1 minutes
Vessel 1: Crane starts unloading container 3 at 18 hours and 1 minutes
Vessel 1: Crane places container 3 on truck at 18 hours and 4 minutes
Truck starts transporting container 3 to yard block at 18 hours and 4 minutes
Truck returns to quay crane at 18 hours and 10 minutes
Vessel 1: Crane starts unloading container 4 a

# Simulation Ended